# seq2seq构建写对联AI
### 代码参考：[seq2seq-couplet](https://github.com/wb14123/seq2seq-couplet)

### 问题背景介绍
对联又称对子，对仗工整，平仄协调，是一字一音的汉文语言独特的艺术形式，是中国传统文化瑰宝。对联的上下联有着非常工整的对应关系，我们可以尝试使用神经网络学习对应关系，进而完成对对联任务，而之前提到的seq2seq模型，是非常典型的序列映射学习模型，可以在本场景下使用。

![](../img/couplet.jpeg)

### seq2seq对对联
##### \[稀牛学院 x 网易云课程\]《AI工程师(自然语言处理方向)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

这里构建的对对联AI应用也是seq2seq模型，使用的就是我们在上一门中讲解到的模型。

![](../img/[1]_seq2seq_1.gif)

![](../img/[8]_seq2seq_8.gif)

![](../img/attention_tensor_dance.gif)

## 数据读取

In [1]:
from queue import Queue
from threading import Thread
import random


def padding_seq(seq):
    """padding每个输入sequence为最大的sequence长度
    arg：seq of ids
    return: results, padding到max_len的id list
    """
    results = []
    max_len = 0
    for s in seq:
        if max_len < len(s):
            max_len = len(s)
    for i in range(0, len(seq)):
        l = max_len - len(seq[i])
        results.append(seq[i] + [0 for j in range(l)])
    return results


def encode_text(words, vocab_indices):
    """把文本序列映射为id序列
    args: words, 输入对联中每个字组成的list
          vocab_indices，词到id的dict
    return：文本序列对应的id序列      
    """
    return [vocab_indices[word] for word in words if word in vocab_indices]


def decode_text(labels, vocabs, end_token='</s>'):
    """把id序列映射为文本序列
    args: labels, decoder输出的预测结果list
          vocab，id到词的dict
    return：results，' '连接的预测文本
    """
    results = []
    for idx in labels:
        word = vocabs[idx]
        if word == end_token:
            return ' '.join(results)
        results.append(word)
    return ' '.join(results)


def read_vocab(vocab_file):
    """读取词表文件
    return：vocabs，list包含文件中的所有字及<s>,</s>,','
    """
    f = open(vocab_file, 'rb')
    vocabs = [line.decode('utf-8')[:-1] for line in f]
    f.close()
    return vocabs


class SeqReader():
    """输入序列读取类"""
    def __init__(self,
                 input_file,
                 target_file,
                 vocab_file,
                 batch_size,
                 queue_size=2048,
                 worker_size=2,
                 end_token='</s>',
                 padding=True,
                 max_len=50):
        self.input_file = input_file
        self.target_file = target_file
        self.vocabs = read_vocab(vocab_file)
        # 词到id的dict
        self.vocab_indices = dict((c, i) for i, c in enumerate(self.vocabs))
        self.batch_size = batch_size
        self.padding = padding      
        self.data_queue = Queue(queue_size)
        self.worker_size = worker_size
        self.end_token = end_token
        self.max_len = max_len
        
        with open(self.input_file, 'rb') as f:
            for i, line in enumerate(f):
                pass
            f.close()
            self.single_lines = i + 1    # 输入文件总行数
            
        self.data_size = int(self.single_lines / batch_size) # batch总数
        self.data_pos = 0     # 指针，self.data中的某一个索引
        self._init_reader()

    def start(self):
        """多线程运行_init_reader()"""
        for i in range(self.worker_size):
            t = Thread(target=self._init_reader())
            t.daemon = True   # 守护线程，后台运行
            t.start()
        return
        
    def read_single_data(self):
        """读取一组数据，
        return:{
                'in_seq': in_seq,
                'in_seq_len': len(in_seq),
                'target_seq': target_seq,
                'target_seq_len': len(target_seq) - 1
        }
        """
        if self.data_pos >= len(self.data):
            random.shuffle(self.data)
            self.data_pos = 0
        result = self.data[self.data_pos]
        self.data_pos += 1
        return result

    def read(self):
        """batch生成器
        yield：batch，dict类型，{
                'in_seq': [[seq1], [seq2], ...],
                'in_seq_len': [int, int, ...],
                'target_seq': [[seq1], [seq2], ...],
                'target_seq_len': [int, int, ...]
                }
        """
        while True:
            batch = {
                'in_seq': [],
                'in_seq_len': [],
                'target_seq': [],
                'target_seq_len': []
            }
            for i in range(0, self.batch_size):
                item = self.read_single_data()
                batch['in_seq'].append(item['in_seq'])
                batch['in_seq_len'].append(item['in_seq_len'])
                batch['target_seq'].append(item['target_seq'])
                batch['target_seq_len'].append(item['target_seq_len'])
            if self.padding:
                batch['in_seq'] = padding_seq(batch['in_seq'])
                batch['target_seq'] = padding_seq(batch['target_seq'])
            yield batch

    def _init_reader(self):
        """文件读取，预处理数据格式
        self.data保存了转化为id的每组input sequence、target sequence的dict，储
        存在list中
        """
        self.data = []   # 初始化输出数据为空list
        input_f = open(self.input_file, 'rb')
        target_f = open(self.target_file, 'rb')
        
        for input_line in input_f:
            input_line = input_line.decode('utf-8')[:-1]
            # target_line按行读取
            target_line = target_f.readline().decode('utf-8')[:-1]
            # 文本以' '为每个字的分隔符 
            input_words = [x for x in input_line.split(' ') if x != '']
            
            if len(input_words) >= self.max_len:
                input_words = input_words[:self.max_len - 1]
            input_words.append(self.end_token)
            
            target_words = [x for x in target_line.split(' ') if x != '']
            if len(target_words) >= self.max_len:
                target_words = target_words[:self.max_len - 1]
            target_words = ['<s>',] + target_words # 加入开始符
            target_words.append(self.end_token)    # 加入结束符
            
            in_seq = encode_text(input_words, self.vocab_indices)
            target_seq = encode_text(target_words, self.vocab_indices)
            self.data.append({
                'in_seq': in_seq,
                'in_seq_len': len(in_seq),
                'target_seq': target_seq,
                'target_seq_len': len(target_seq) - 1 # <s>不计入
            })
            
        input_f.close()
        target_f.close()
        self.data_pos = len(self.data)

## 评估函数

In [2]:
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Python implementation of BLEU and smooth-BLEU.

This module provides a Python implementation of BLEU and smooth-BLEU.
Smooth BLEU is computed following the method outlined in the paper:
Chin-Yew Lin, Franz Josef Och. ORANGE: a method for evaluating automatic
evaluation metrics for machine translation. COLING 2004.
"""

import collections
import math


def _get_ngrams(segment, max_order):
    """Extracts all n-grams upto a given maximum order from an input segment.

    Args:
        segment: text segment from which n-grams will be extracted.
        max_order: maximum length in tokens of the n-grams returned by this
            methods.

    Returns:
        The Counter containing all n-grams upto max_order in segment
        with a count of how many times each n-gram occurred.
        (all n-grams upto max_order)，keys：n-gram，value：count
    """
    ngram_counts = collections.Counter()
    for order in range(1, max_order + 1):
        for i in range(0, len(segment) - order + 1):
            ngram = tuple(segment[i:i + order])
            ngram_counts[ngram] += 1
    return ngram_counts


def compute_bleu(reference_corpus,
                 translation_corpus,
                 max_order=4,
                 smooth=False):
    """Computes BLEU score of translated segments against one or more references.

    Args:
        reference_corpus: list of lists of references for each translation. Each
            reference should be tokenized into a list of tokens.
        translation_corpus: list of translations to score. Each translation
            should be tokenized into a list of tokens.
        max_order: Maximum n-gram order to use when computing BLEU score.
        smooth: Whether or not to apply Lin et al. 2004 smoothing.

    Returns:
        3-Tuple with the BLEU score, n-gram precisions, geometric mean of n-gram
        precisions and brevity penalty.
        bleu：float，翻译句子的bleu得分, 
        precisions：list, 包含每种ngram的准确率，
        bp：brevity penalty, 短句惩罚系数，
        ratio：translation_length / min(reference_length), 
        translation_length：int，翻译长度, 
        reference_length：int，最短的reference长度
    """
    matches_by_order = [0] * max_order
    possible_matches_by_order = [0] * max_order
    reference_length = 0
    translation_length = 0
    
    for (references, translation) in zip(reference_corpus, translation_corpus):
        reference_length += min(len(r) for r in references)
        translation_length += len(translation)

        merged_ref_ngram_counts = collections.Counter()
        # 同时考虑多个references
        for reference in references:
            merged_ref_ngram_counts |= _get_ngrams(reference, max_order)# 位或
        translation_ngram_counts = _get_ngrams(translation, max_order)
        overlap = translation_ngram_counts & merged_ref_ngram_counts    # 位与
        # matches_by_order：｛len(ngram)：sum of counts｝
        for ngram in overlap:
            matches_by_order[len(ngram) - 1] += overlap[ngram]
        # possible_matches_by_order(可匹配n-gram总数)：
        # ｛len(ngram)：sum of each ngram｝
        for order in range(1, max_order + 1):
            possible_matches = len(translation) - order + 1
            if possible_matches > 0:
                possible_matches_by_order[order - 1] += possible_matches

    precisions = [0] * max_order
    for i in range(0, max_order):
        if smooth:
            precisions[i] = ((matches_by_order[i] + 1.) /
                             (possible_matches_by_order[i] + 1.))
        else:
            if possible_matches_by_order[i] > 0:
                precisions[i] = (
                    float(matches_by_order[i]) / possible_matches_by_order[i])
            else:
                precisions[i] = 0.0

    if min(precisions) > 0:
        p_log_sum = sum((1. / max_order) * math.log(p) for p in precisions)
        geo_mean = math.exp(p_log_sum)
    else:
        geo_mean = 0

    # 翻译长度惩罚（对较短的翻译基于较大的惩罚，以防止短翻译准确率会更高的问题）
    ratio = float(translation_length) / reference_length
    if ratio > 1.0:
        bp = 1.
    else:
        bp = math.exp(1 - 1. / ratio)

    bleu = geo_mean * bp

    return (bleu, precisions, bp, ratio, translation_length, reference_length)

## 定义seq2seq

In [3]:
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.python.layers import core as layers_core

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
def getLayeredCell(layer_size,
                   num_units,
                   input_keep_prob,
                   output_keep_prob=1.0):
    '''多层rnn单元构造'''
    return rnn.MultiRNNCell([
        rnn.DropoutWrapper(
            tf.nn.rnn_cell.LSTMCell(
                name='basic_lstm_cell', num_units=num_units), input_keep_prob,
            output_keep_prob) for i in range(layer_size)
    ])


def bi_encoder(embed_input, in_seq_len, num_units, layer_size,
               input_keep_prob):
    '''双向rnn编码器
    embed_input：embeddirding后的输入序列
    num_units：隐藏层单元数
    layer_size：rnn层数
    return: 
      bidirectional_dynamic_rnn不同于bidirectional_rnn，结果级联分层输出，可
      concat到一起
      encoder_output: 每个timestep输出，每层输出按最后一维concat到一起
      encoder_state：每层的final state，(output_state_fw, output_state_bw)
    '''
    # encode input into a vector
    bi_layer_size = int(layer_size / 2)
    encode_cell_fw = getLayeredCell(bi_layer_size, num_units, input_keep_prob)
    encode_cell_bw = getLayeredCell(bi_layer_size, num_units, input_keep_prob)
    bi_encoder_output, bi_encoder_state = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=encode_cell_fw,
        cell_bw=encode_cell_bw,
        inputs=embed_input,
        sequence_length=in_seq_len,
        dtype=embed_input.dtype,
        time_major=False)

    # concat encode output and state
    encoder_output = tf.concat(bi_encoder_output, -1)
    encoder_state = []
    for layer_id in range(bi_layer_size):
        encoder_state.append(bi_encoder_state[0][layer_id])
        encoder_state.append(bi_encoder_state[1][layer_id])
    encoder_state = tuple(encoder_state)
    return encoder_output, encoder_state


def attention_decoder_cell(encoder_output, in_seq_len, num_units, layer_size,
                           input_keep_prob):
    '''attention decoder
    return: 加入attention_mechanim的decoder cell
    '''
    attention_mechanim = tf.contrib.seq2seq.BahdanauAttention(
        num_units, encoder_output, in_seq_len, normalize=True)
    # attention_mechanim = tf.contrib.seq2seq.LuongAttention(num_units,
    #         encoder_output, in_seq_len, scale = True)
    cell = getLayeredCell(layer_size, num_units, input_keep_prob)
    cell = tf.contrib.seq2seq.AttentionWrapper(
        cell, attention_mechanim, attention_layer_size=num_units)
    return cell


def decoder_projection(output, output_size):
    return tf.layers.dense(
        output,
        output_size,
        activation=None,
        use_bias=False,
        name='output_mlp')


def train_decoder(encoder_output, in_seq_len, target_seq, target_seq_len,
                  encoder_state, num_units, layers, embedding, output_size,
                  input_keep_prob, projection_layer):
    '''只进行train过程'''
    decoder_cell = attention_decoder_cell(encoder_output, in_seq_len,
                                          num_units, layers, input_keep_prob)
    batch_size = tf.shape(in_seq_len)[0]
    init_state = decoder_cell.zero_state(
        batch_size, tf.float32).clone(cell_state=encoder_state)
    helper = tf.contrib.seq2seq.TrainingHelper(
        target_seq, target_seq_len, time_major=False)
    decoder = tf.contrib.seq2seq.BasicDecoder(
        decoder_cell, helper, init_state, output_layer=projection_layer)
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        decoder, maximum_iterations=100)
    return outputs.rnn_output


def infer_decoder(encoder_output, in_seq_len, encoder_state, num_units, layers,
                  embedding, output_size, input_keep_prob, projection_layer):
    '''seq2seq函数中可以使用beamsearch方法来进行decoder'''
    decoder_cell = attention_decoder_cell(encoder_output, in_seq_len,
                                          num_units, layers, input_keep_prob)

    batch_size = tf.shape(in_seq_len)[0]
    init_state = decoder_cell.zero_state(
        batch_size, tf.float32).clone(cell_state=encoder_state)

    # TODO: start tokens and end tokens are hard code
    """
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
            embedding, tf.fill([batch_size], 0), 1)
    decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper,
            init_state, output_layer=projection_layer)
    """

    decoder = tf.contrib.seq2seq.BeamSearchDecoder(
        cell=decoder_cell,
        embedding=embedding,
        start_tokens=tf.fill([batch_size], 0),
        end_token=1,
        initial_state=init_state,
        beam_width=10,
        output_layer=projection_layer,
        length_penalty_weight=1.0)

    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        decoder, maximum_iterations=100)
    return outputs.sample_id


def seq2seq(in_seq, in_seq_len, target_seq, target_seq_len, vocab_size,
            num_units, layers, dropout):
    """seq2seq模型建立
    return: training -- outputs.rnn_output: rnn输出的概率分布结果
            infering -- outputs.sample_id：输出词id
    """
    in_shape = tf.shape(in_seq)
    batch_size = in_shape[0]
    
    # 训练开启dropout，预测不开启
    if target_seq != None:
        input_keep_prob = 1 - dropout
    else:
        input_keep_prob = 1
    
    # 全连接层输出预测
    projection_layer = layers_core.Dense(vocab_size, use_bias=False)

    # embedding input and target sequence
    with tf.device('/cpu:0'):
        embedding = tf.get_variable(
            name='embedding', shape=[vocab_size, num_units])
    embed_input = tf.nn.embedding_lookup(embedding, in_seq, name='embed_input')

    # encode and decode
    encoder_output, encoder_state = bi_encoder(
        embed_input, in_seq_len, num_units, layers, input_keep_prob)

    decoder_cell = attention_decoder_cell(encoder_output, in_seq_len,
                                          num_units, layers, input_keep_prob)
    batch_size = tf.shape(in_seq_len)[0]
    # decoder初始化，权重初始化，并且将cell state初始化为encoder的final state
    init_state = decoder_cell.zero_state(
        batch_size, tf.float32).clone(cell_state=encoder_state)

    if target_seq != None:
        embed_target = tf.nn.embedding_lookup(
            embedding, target_seq, name='embed_target')
        helper = tf.contrib.seq2seq.TrainingHelper(
            embed_target, target_seq_len, time_major=False)
    else:
        # TODO: start tokens and end tokens are hard code
        # 0，1分别应对应句子的起始符id和终止符id
        helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
            embedding, tf.fill([batch_size], 0), 1)
        
    decoder = tf.contrib.seq2seq.BasicDecoder(
        decoder_cell, helper, init_state, output_layer=projection_layer)
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        decoder, maximum_iterations=100)
    
    if target_seq != None:
        return outputs.rnn_output
    else:
        return outputs.sample_id


def seq_loss(output, target, seq_len):
    '''计算损失
    target：包括一个起始符
    '''
    target = target[:, 1:]
    cost = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=output, labels=target)
    batch_size = tf.shape(target)[0]
    # 不每个句子对都达到max_timestep，排除多余字符带来的损失
    loss_mask = tf.sequence_mask(seq_len, tf.shape(output)[1])
    cost = cost * tf.to_float(loss_mask)
    return tf.reduce_sum(cost) / tf.to_float(batch_size)

## 模型定义

In [5]:
import tensorflow as tf
from os import path
import random


class Model():
    """创建模型类"""
    def __init__(self,
                 train_input_file,
                 train_target_file,
                 test_input_file,
                 test_target_file,
                 vocab_file,
                 num_units,
                 layers,
                 dropout,
                 batch_size,
                 learning_rate,
                 output_dir,
                 save_step=100,
                 eval_step=1000,
                 param_histogram=False,
                 restore_model=False,
                 init_train=True,
                 init_infer=False):
        self.num_units = num_units  # 单个RNN结构中，神经元数目
        self.layers = layers
        self.dropout = dropout
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.save_step = save_step
        self.eval_step = eval_step
        self.param_histogram = param_histogram
        self.restore_model = restore_model  # boolen
        self.init_train = init_train
        self.init_infer = init_infer

        if init_train:
            self.train_reader = SeqReader(train_input_file, train_target_file,
                                          vocab_file, batch_size)
            self.train_reader.start()  # 多线程
            self.train_data = self.train_reader.read()  # yield batch
            self.eval_reader = SeqReader(test_input_file, test_target_file,
                                         vocab_file, batch_size)
            self.eval_reader.start()
            self.eval_data = self.eval_reader.read()

        self.model_file = path.join(output_dir, 'model.ckpl')
        self.log_writter = tf.summary.FileWriter(output_dir)

        if init_train:
            self._init_train()
            self._init_eval()

        if init_infer:
            self.infer_vocabs = reader.read_vocab(vocab_file)
            self.infer_vocab_indices = dict(
                (c, i) for i, c in enumerate(self.infer_vocabs))
            self._init_infer()
            self.reload_infer_model()

    def gpu_session_config(self):
        # allow_growth: 刚一开始分配少量的GPU容量，然后按需慢慢的增加
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        return config

    def _init_train(self):
        '''初始化训练会话'''
        self.train_graph = tf.Graph()
        with self.train_graph.as_default():
            # 输入
            self.train_in_seq = tf.placeholder(
                tf.int32, shape=[self.batch_size, None])
            self.train_in_seq_len = tf.placeholder(
                tf.int32, shape=[self.batch_size])
            self.train_target_seq = tf.placeholder(
                tf.int32, shape=[self.batch_size, None])
            self.train_target_seq_len = tf.placeholder(
                tf.int32, shape=[self.batch_size])
            
            # 输出
            output = seq2seq(self.train_in_seq, self.train_in_seq_len,
                             self.train_target_seq, self.train_target_seq_len,
                             len(self.train_reader.vocabs), self.num_units,
                             self.layers, self.dropout)
            self.train_output = tf.argmax(tf.nn.softmax(output), 2)
            
            # 损失
            self.loss = seq_loss(output, self.train_target_seq,
                                 self.train_target_seq_len)
            
            # 梯度截断
            params = tf.trainable_variables()
            gradients = tf.gradients(self.loss, params)
            clipped_gradients, _ = tf.clip_by_global_norm(gradients, 0.5)
            self.train_op = tf.train.AdamOptimizer(
                learning_rate=self.learning_rate).apply_gradients(
                    list(zip(clipped_gradients, params)))
            
            # 变量统计输出历史变化情况
            if self.param_histogram:
                for v in tf.trainable_variables():
                    tf.summary.histogram('train_' + v.name, v)
            tf.summary.scalar('loss', self.loss)
            self.train_summary = tf.summary.merge_all()
            
            self.train_init = tf.global_variables_initializer()
            self.train_saver = tf.train.Saver()
        
        self.train_session = tf.Session(
            graph=self.train_graph, config=self.gpu_session_config())

    def _init_eval(self):
        '''初始化测试操作'''
        self.eval_graph = tf.Graph()
        with self.eval_graph.as_default():
            
            self.eval_in_seq = tf.placeholder(
                tf.int32, shape=[self.batch_size, None])
            self.eval_in_seq_len = tf.placeholder(
                tf.int32, shape=[self.batch_size])
            
            self.eval_output = seq2seq(
                self.eval_in_seq, self.eval_in_seq_len, None, None,
                len(self.eval_reader.vocabs), self.num_units, self.layers,
                self.dropout)
            
            if self.param_histogram:
                for v in tf.trainable_variables():
                    tf.summary.histogram('eval_' + v.name, v)
            self.eval_summary = tf.summary.merge_all()
            self.eval_saver = tf.train.Saver()
        
        self.eval_session = tf.Session(
            graph=self.eval_graph, config=self.gpu_session_config())

    def _init_infer(self):
        '''初始化推断'''
        self.infer_graph = tf.Graph()
        with self.infer_graph.as_default():
            self.infer_in_seq = tf.placeholder(tf.int32, shape=[1, None])
            self.infer_in_seq_len = tf.placeholder(tf.int32, shape=[1])
            self.infer_output = seq2seq(self.infer_in_seq,
                                        self.infer_in_seq_len, None, None,
                                        len(self.infer_vocabs), self.num_units,
                                        self.layers, self.dropout)
            
            self.infer_saver = tf.train.Saver()
        
        self.infer_session = tf.Session(
            graph=self.infer_graph, config=self.gpu_session_config())

    def train(self, epochs, start=0):
        if not self.init_train:
            raise Exception('Train graph is not inited!')
        
        with self.train_graph.as_default():
            if path.isfile(self.model_file + '.meta') and self.restore_model:
                print("Reloading model file before training.")
                self.train_saver.restore(self.train_session, self.model_file)
            else:
                self.train_session.run(self.train_init)
            
            total_loss = 0
            for step in range(start, epochs):
                data = next(self.train_data) # yeild
                in_seq = data['in_seq']
                in_seq_len = data['in_seq_len']
                target_seq = data['target_seq']
                target_seq_len = data['target_seq_len']
                
                output, loss, train, summary = self.train_session.run(
                    [
                        self.train_output, 
                        self.loss, 
                        self.train_op,
                        self.train_summary
                    ],
                    feed_dict={
                        self.train_in_seq: in_seq,
                        self.train_in_seq_len: in_seq_len,
                        self.train_target_seq: target_seq,
                        self.train_target_seq_len: target_seq_len
                    })
                
                total_loss += loss
                
                self.log_writter.add_summary(summary, step)
                
                if step % self.save_step == 0:
                    self.train_saver.save(self.train_session, self.model_file)
                    print(("Saving model. Step: %d, loss: %f" %
                           (step, total_loss / self.save_step)))
                    # print sample output
                    sid = random.randint(0, self.batch_size - 1)
                    input_text = decode_text(in_seq[sid],
                                                    self.eval_reader.vocabs)
                    output_text = decode_text(output[sid],
                                                     self.train_reader.vocabs)
                    target_text = decode_text(
                        target_seq[sid],
                        self.train_reader.vocabs).split(' ')[1:]
                    target_text = ' '.join(target_text)
                    print('******************************')
                    print(('src: ' + input_text))
                    print(('output: ' + output_text))
                    print(('target: ' + target_text))
                    
                if step % self.eval_step == 0:
                    bleu_score = self.eval(step)
                    print(("Evaluate model. Step: %d, score: %f, loss: %f" %
                           (step, bleu_score, total_loss / self.save_step)))
                    eval_summary = tf.Summary(value=[
                        tf.Summary.Value(tag='bleu', simple_value=bleu_score)
                    ])
                    self.log_writter.add_summary(eval_summary, step)
                
                if step % self.save_step == 0:
                    total_loss = 0

    def eval(self, train_step):
        '''测试函数，bleu_score'''
        with self.eval_graph.as_default():
            self.eval_saver.restore(self.eval_session, self.model_file)
            bleu_score = 0
            target_results = []
            output_results = []
            for step in range(0, self.eval_reader.data_size):
                data = next(self.eval_data)
                in_seq = data['in_seq']
                in_seq_len = data['in_seq_len']
                target_seq = data['target_seq']
                target_seq_len = data['target_seq_len']
                outputs = self.eval_session.run(
                    self.eval_output,
                    feed_dict={
                        self.eval_in_seq: in_seq,
                        self.eval_in_seq_len: in_seq_len
                    })
                
                for i in range(len(outputs)):
                    output = outputs[i]
                    target = target_seq[i]
                    output_text = decode_text(
                        output, self.eval_reader.vocabs).split(' ')
                    target_text = decode_text(
                        target[1:], self.eval_reader.vocabs).split(' ')
                    
                    prob = int(
                        self.eval_reader.data_size * self.batch_size / 10)
                    
                    target_results.append([target_text])
                    output_results.append(output_text)
                    # 随机输出结果
                    if random.randint(1, prob) == 1:
                        print('====================')
                        input_text = decode_text(
                            in_seq[i], self.eval_reader.vocabs)
                        print(('src:' + input_text))
                        print(('output: ' + ' '.join(output_text)))
                        print(('target: ' + ' '.join(target_text)))
                        
            return compute_bleu(target_results, output_results)[0] * 100

    def reload_infer_model(self):
        # 重新加载推理图
        with self.infer_graph.as_default():
            self.infer_saver.restore(self.infer_session, self.model_file)

    def infer(self, text):
        if not self.init_infer:
            raise Exception('Infer graph is not inited!')
            
        with self.infer_graph.as_default():
            in_seq = encode_text(
                text.split(' ') + [
                    '</s>',
                ], self.infer_vocab_indices)
            in_seq_len = len(in_seq)
            
            outputs = self.infer_session.run(
                self.infer_output,
                feed_dict={
                    self.infer_in_seq: [in_seq],
                    self.infer_in_seq_len: [in_seq_len]
                })
            output = outputs[0]
            output_text = decode_text(output, self.infer_vocabs)
            return output_text

## 模型训练

In [6]:
m = Model(
        './couplet/train/in.txt',
        './couplet/train/out.txt',
        './couplet/test/in.txt',
        './couplet/test/out.txt',
        './couplet/vocabs',
        num_units=256,
        layers=4,
        dropout=0.2,
        batch_size=32,
        learning_rate=0.001,
        output_dir='./models/output_couplet',
        restore_model=False
)

m.train(5000000)

Saving model. Step: 0, loss: 1.017375
******************************
src: 龙 根 在 抱 ， 基 业 于 胸 ， 发 展 进 行 时 ， 须 知 岁 月 无 期 、 资 源 有 限
output: 牿 帮 帮 酼 酼 酼 酼 酼 邀 酼 灣 灣 灣 灣 灣 灣 灣 灣 灣 灣 灣 灣 灣 灣 帮 帮 帮 帮
target: 红 线 铭 心 ， 蓝 图 入 梦 ， 和 谐 规 划 处 ， 尤 记 尺 疆 易 失 、 寸 土 难 还
INFO:tensorflow:Restoring parameters from ./models/output_couplet\model.ckpl
src:千 里 青 青 草 ， 碧 色 连 天 ， 无 边 春 色 共 谁 看
output: 
target: 一 池 淡 淡 荷 ， 清 香 扑 面 ， 几 度 花 香 伴 月 眠
src:青 山 随 我 隐
output: 
target: 碧 水 为 君 酬
src:治 水 禹 王 功 ， 看 龙 伏 波 平 ， 赫 赫 千 秋 歌 大 业
output: 
target: 富 民 政 策 好 ， 喜 鱼 香 稻 熟 ， 熙 熙 百 姓 乐 丰 年
src:苦 楚
output: 
target: 艰 难
src:雨 前 茶 绿 宜 烹 雪
output: 
target: 宴 罢 酒 高 莫 驾 车
src:以 善 结 禅 缘 ， 诚 通 佛 性 ， 人 和 兼 地 利
output: 
target: 任 山 扬 龙 首 ， 水 唱 凤 歌 ， 雨 顺 更 风 调
src:万 里 香 风 菩 萨 道
output: 
target: 一 声 梵 唱 华 严 经
src:释 怀 心 见 佛
output: 
target: 难 道 不 如 人
src:孝 容 海 岳 ， 古 邑 人 文 尤 焕 彩
output: 
target: 义 薄 云 天 ， 新 时 山 水 更 添 娇
Evaluate model. Step: 0, score: 0.000000, loss: 1.017375
Saving model. Step: 100, loss: 71.098539
******************

KeyboardInterrupt: 